# OLS Regression on Rental Data and Geospacial Data

In [75]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.preprocessing import PolynomialFeatures, StandardScaler


In [76]:
# Load the CSV file into a Pandas DataFrame
data1 = pd.read_csv('4_Final_Data.csv')

# Display the DataFrame
data1


,Latitude,Longitude,Monthly rent,Aconto,Square meters,Rooms,Furnished,Shareable,Pets allowed,Elevator,...,Floor_-1 to 0,Floor_1-3,Floor_3-8,Floor_9 or above,Distance to Transport Station (km),Distance to Beach (km),Distance to School (km),Distance to Restaurant (km),Distance to Hospital (km),Distance to Mall (km)
0,55.393856,10.394185,5400.0,800.0,62.0,2.0,0.0,0.0,0.0,1.0,...,0,1,0,0,0.227327,10.937984,0.347042,0.114848,1.856833,0.972492
1,56.300211,10.477949,7100.0,700.0,100.0,4.0,0.0,0.0,1.0,0.0,...,0,1,0,0,6.490199,5.187801,0.198759,0.081711,22.428327,20.441833
2,57.057390,9.936155,5800.0,300.0,64.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,2.057982,46.063274,0.894998,1.091345,1.726659,1.252278
3,55.721333,12.556324,8200.0,500.0,43.0,1.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.684702,2.087369,0.121870,0.253656,0.873095,1.532133
4,56.199538,10.198303,8195.0,600.0,60.0,3.0,0.0,1.0,1.0,1.0,...,0,1,0,0,1.569096,12.856874,0.929082,0.441412,1.803299,0.677731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9962,57.053293,9.911053,7600.0,850.0,73.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.158809,45.438610,0.556684,0.239033,0.530896,0.832760
9963,55.485419,9.500225,8250.0,1000.0,84.0,3.0,0.0,0.0,1.0,1.0,...,0,1,0,0,1.342171,1.674266,1.353033,0.310371,1.846679,4.283248
9964,56.449611,9.404525,9500.0,1550.0,130.0,4.0,0.0,0.0,0.0,0.0,...,0,0,1,0,0.466744,33.988380,0.126922,0.030089,0.458502,0.100810
9965,55.307771,11.543354,7150.0,500.0,87.0,3.0,0.0,0.0,0.0,0.0,...,0,1,0,0,12.496495,10.451021,2.480094,8.383154,15.404188,16.237640


In [77]:
# Split the data into a 70% training set and a 30% testing set
train_data, test_data = train_test_split(data1, test_size=0.3, random_state=42)

data = train_data

#Display the data used for training
data

,Latitude,Longitude,Monthly rent,Aconto,Square meters,Rooms,Furnished,Shareable,Pets allowed,Elevator,...,Floor_-1 to 0,Floor_1-3,Floor_3-8,Floor_9 or above,Distance to Transport Station (km),Distance to Beach (km),Distance to School (km),Distance to Restaurant (km),Distance to Hospital (km),Distance to Mall (km)
468,56.202715,10.235290,5800.0,800.0,38.0,1.0,0.0,0.0,1.0,0.0,...,0,0,1,0,0.522472,12.850379,0.615806,1.155892,4.088385,1.958109
1545,55.478811,12.175062,10500.0,500.0,85.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.120650,2.919823,0.173839,1.927658,0.632126,10.111791
8077,56.369091,8.627981,4700.0,0.0,45.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.626280,12.766357,0.250825,0.658392,28.711652,0.934676
2683,55.441216,11.789264,6900.0,910.0,75.0,3.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0.403884,6.245501,0.725544,0.361684,0.994046,1.189914
9875,57.221161,9.673143,5929.0,276.0,91.0,2.0,0.0,0.0,0.0,0.0,...,0,1,0,0,6.024007,15.160941,1.239277,5.731721,24.076024,6.346106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,56.150558,10.170229,7900.0,650.0,63.0,2.0,0.0,0.0,1.0,1.0,...,1,0,0,0,1.708648,8.546161,0.376386,0.537677,3.823016,1.535872
5191,55.640281,12.583101,14830.0,950.0,91.0,3.0,0.0,0.0,0.0,0.0,...,0,0,0,1,0.246086,1.912752,1.026641,0.313369,0.384578,1.145962
5390,55.714045,9.532166,7443.0,560.0,77.0,3.0,0.0,1.0,0.0,0.0,...,1,0,0,0,0.319052,3.220593,0.730781,0.303420,0.378538,0.852703
860,56.040973,12.569082,14000.0,1000.0,112.0,4.0,0.0,0.0,0.0,1.0,...,0,1,0,0,2.353796,2.121391,0.365182,1.208763,2.409418,2.535417


# Without Geospatial Data

In [78]:
# Prepare the features (X) and the label (y)
# Drop 'Monthly rent', 'Longitude' and 'Latitude' columns
X = data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)'], axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Add a constant to the features (intercept term)
X = sm.add_constant(X)

# Create an OLS model
model = sm.OLS(y, X)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE_train1 = np.sqrt((residuals ** 2).mean())
print('Train Root Mean Squared Error (RMSE):', RMSE_train1)

                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.747
Model:                            OLS   Adj. R-squared:                  0.746
Method:                 Least Squares   F-statistic:                     483.0
Date:                Thu, 17 Aug 2023   Prob (F-statistic):               0.00
Time:                        12:30:55   Log-Likelihood:                -62892.
No. Observations:                6976   AIC:                         1.258e+05
Df Residuals:                    6953   BIC:                         1.260e+05
Df Model:                          22                                         
Covariance Type:                  HC3                                         
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                  1.21e+04 

## Test the Model Without Geospatial Data on the Test Data Set

In [79]:
# Prepare the features in the test data for the first model
X_test1 = test_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0', 'Distance to Transport Station (km)', 'Distance to Beach (km)', 'Distance to School (km)', 'Distance to Restaurant (km)', 'Distance to Hospital (km)', 'Distance to Mall (km)'], axis=1)

# Add a constant to the features (intercept term)
X_test1 = sm.add_constant(X_test1)

# True 'Monthly rent' values in the test data
y_test = test_data['Monthly rent']

# Get the Predicted Values on the test data
y_pred_test1 = results.predict(X_test1)

# Calculate the Residuals for the test data
residuals_test1 = y_test - y_pred_test1

# Calculate the RMSE for the test data
RMSE_test1 = np.sqrt((residuals_test1 ** 2).mean())
print('Test Root Mean Squared Error (RMSE) for First Model:', RMSE_test1)


Test Root Mean Squared Error (RMSE) for First Model: 1968.6581314465182


# With Geospatial Data

In [80]:
# Prepare the features (X) and the label (y)
# Drop 'Monthly rent', 'Longitude' and 'Latitude' columns
X = data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)
y = data['Monthly rent']  # Labels (the 'Monthly rent' column)

# Add a constant to the features (intercept term)
X = sm.add_constant(X)

# Create an OLS model
model = sm.OLS(y, X)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Get the Predicted Values
y_pred = results.predict(X)

# Calculate the Residuals
residuals = y - y_pred

# Calculate the RMSE
RMSE_train2 = np.sqrt((residuals ** 2).mean())
print('Train Root Mean Squared Error (RMSE):', RMSE_train2)



                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.774
Model:                            OLS   Adj. R-squared:                  0.773
Method:                 Least Squares   F-statistic:                     418.4
Date:                Thu, 17 Aug 2023   Prob (F-statistic):               0.00
Time:                        12:30:55   Log-Likelihood:                -62503.
No. Observations:                6976   AIC:                         1.251e+05
Df Residuals:                    6947   BIC:                         1.253e+05
Df Model:                          28                                         
Covariance Type:                  HC3                                         
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
cons

## Test the Model With Geospatial Data on the Test Data Set

In [81]:
# Prepare the features in the test data for the second model
X_test2 = test_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)

# Add a constant to the features (intercept term)
X_test2 = sm.add_constant(X_test2)

# Get the Predicted Values on the test data
y_pred_test2 = results.predict(X_test2)

# Calculate the Residuals for the test data
residuals_test2 = y_test - y_pred_test2

# Calculate the RMSE for the test data
RMSE_test2 = np.sqrt((residuals_test2 ** 2).mean())
print('Test Root Mean Squared Error (RMSE) for Second Model:', RMSE_test2)


Test Root Mean Squared Error (RMSE) for Second Model: 1866.2286299189257


# Test/Train Summary

In [82]:
# Create a dictionary with the RMSE values
rmse_data = {
    'Data Split': ['Train', 'Test', 'Train', 'Test'],
    'Model': ['Without Geospatial Data', 'Without Geospatial Data', 'With Geospatial Data', 'With Geospatial Data'],
    'RMSE': [RMSE_train1, RMSE_test1, RMSE_train2, RMSE_test2]
}

# Convert the dictionary to a Pandas DataFrame
rmse_df = pd.DataFrame(rmse_data)

# Print the DataFrame as a table
print(rmse_df)


  Data Split                    Model         RMSE
0      Train  Without Geospatial Data  1991.298406
1       Test  Without Geospatial Data  1968.658131
2      Train     With Geospatial Data  1883.234721
3       Test     With Geospatial Data  1866.228630


In [83]:
# Extract columns that are related to postal codes
postal_code_columns = [col for col in train_data.columns if 'PostalCode' in col]

# Create interaction terms only between postal codes and square meters
for postal_col in tqdm(postal_code_columns, desc='Creating Interaction Terms'):  # tqdm function creates the progress bar
    interaction_col_name = f'{postal_col}_x_Square meters'  # name of the interaction column
    train_data[interaction_col_name] = train_data[postal_col] * train_data['Square meters']
    test_data[interaction_col_name] = test_data[postal_col] * test_data['Square meters']

# Prepare the features (X) and the label (y) using training data
X_train = train_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)
y_train = train_data['Monthly rent']

# Add a constant to the features (intercept term)
X_train = sm.add_constant(X_train)

# Create an OLS model
model = sm.OLS(y_train, X_train)

# Fit the OLS model with heteroscedasticity robust errors
results = model.fit(cov_type='HC3')

# Print the summary of the regression
print(results.summary())

# Use the model to make predictions on the training set
y_pred_train = results.predict(X_train)

# Calculate the Residuals for the training data
residuals_train = y_train - y_pred_train

# Calculate the RMSE for the training data
RMSE_train = np.sqrt((residuals_train ** 2).mean())
print('Train Root Mean Squared Error (RMSE):', RMSE_train)

# Prepare the features (X_test) and the label (y_test) using testing data
X_test = test_data.drop(['Monthly rent', 'Longitude', 'Latitude', 'PostalCode_1000-1999', 'Floor_-1 to 0'], axis=1)
y_test = test_data['Monthly rent']

# Add a constant to the features (intercept term for test data)
X_test = sm.add_constant(X_test)

# Use the model to make predictions on the test set
y_pred_test = results.predict(X_test)

# Calculate the Residuals for the test data
residuals_test = y_test - y_pred_test

# Calculate the RMSE for the test data
RMSE_test = np.sqrt((residuals_test ** 2).mean())
print('Test Root Mean Squared Error (RMSE):', RMSE_test)

Creating Interaction Terms: 100%|██████████| 9/9 [00:00<00:00, 1953.16it/s]

                            OLS Regression Results                            
Dep. Variable:           Monthly rent   R-squared:                       0.810
Model:                            OLS   Adj. R-squared:                  0.809
Method:                 Least Squares   F-statistic:                     521.9
Date:                Thu, 17 Aug 2023   Prob (F-statistic):               0.00
Time:                        12:30:55   Log-Likelihood:                -61890.
No. Observations:                6976   AIC:                         1.239e+05
Df Residuals:                    6939   BIC:                         1.241e+05
Df Model:                          36                                         
Covariance Type:                  HC3                                         
                                           coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------



/Users/philiprefslund/opt/anaconda3/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 37, but rank is 36
  warnings.warn('covariance of constraints does not have full '
